In [2]:
import habitat_sim
import magnum as mn
import warnings
from habitat.tasks.rearrange.rearrange_sim import RearrangeSim
warnings.filterwarnings('ignore')
from habitat_sim.utils.settings import make_cfg
from matplotlib import pyplot as plt
from habitat_sim.utils import viz_utils as vut
from omegaconf import DictConfig
import numpy as np
from habitat.articulated_agents.robots import FetchRobot
from habitat.config.default import get_agent_config
from habitat.config.default_structured_configs import ThirdRGBSensorConfig, HeadRGBSensorConfig, HeadDepthSensorConfig, HeadPanopticSensorConfig
from habitat.config.default_structured_configs import SimulatorConfig, HabitatSimV0Config, AgentConfig
from habitat.config.default import get_agent_config
import habitat
from habitat_sim.physics import JointMotorSettings, MotionType
from omegaconf import OmegaConf

from habitat.config.default_structured_configs import HumanoidJointActionConfig, HumanoidPickActionConfig
from habitat_llm.agent.env.scene import SceneParser
from habitat.config.default_structured_configs import TaskConfig, EnvironmentConfig, DatasetConfig, HabitatConfig
from habitat.config.default_structured_configs import ArmActionConfig, BaseVelocityActionConfig, OracleNavActionConfig
from habitat.core.env import Env
from habitat_llm.perception import Perception


<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


pybullet build time: Oct 25 2023 16:25:40


In [1]:
# Make simulator
def make_sim_cfg(agent_dict):
    # Start the scene config
    sim_cfg = SimulatorConfig(type="RearrangeSim-v0")
    
    # This is for better graphics
    sim_cfg.habitat_sim_v0.enable_hbao = True
    sim_cfg.habitat_sim_v0.enable_physics = True


    sim_cfg.additional_object_paths = [
        "data/objects/ycb/configs/"
    ]

    
    cfg = OmegaConf.create(sim_cfg)

    # Set the scene agents
    cfg.agents = agent_dict
    cfg.agents_order = list(cfg.agents.keys())
    return cfg

def make_hab_cfg(agent_dict, action_dict):
    sim_cfg = make_sim_cfg(agent_dict)
    task_cfg = TaskConfig(type="RearrangeEmptyTask-v0")
    task_cfg.actions = action_dict
    env_cfg = EnvironmentConfig()
    dataset_cfg = DatasetConfig(type="RearrangeDataset-v0", data_path="data/datasets/hssd_test/hssd_ycb_llm_2.json.gz")
    
    
    hab_cfg = HabitatConfig()
    hab_cfg.environment = env_cfg
    hab_cfg.task = task_cfg
    hab_cfg.dataset = dataset_cfg
    hab_cfg.simulator = sim_cfg
    hab_cfg.simulator.seed = hab_cfg.seed

    return hab_cfg

def init_rearrange_env(agent_dict, action_dict):
    hab_cfg = make_hab_cfg(agent_dict, action_dict)
    res_cfg = OmegaConf.create(hab_cfg)
    return Env(res_cfg)

In [3]:
# Define the agent configuration
main_agent_config = AgentConfig()
urdf_path = "data/humanoids/humanoid_data/female_0//female_0.urdf"
main_agent_config.articulated_agent_urdf = urdf_path
main_agent_config.articulated_agent_type = "KinematicHumanoid"
main_agent_config.motion_data_path = "data/humanoids/humanoid_data/female_0//female_0_motion_data_smplx.pkl"


# Define sensors that will be attached to this agent, here a third_rgb sensor and a head_rgb.
# We will later talk about why giving the sensors these names
main_agent_config.sim_sensors = {
    "third_rgb": ThirdRGBSensorConfig(),
    "head_rgb": HeadRGBSensorConfig(),
    "head_depth": HeadDepthSensorConfig(normalize_depth=False),
    "head_panoptic": HeadPanopticSensorConfig(),
}

# We create a dictionary with names of agents and their corresponding agent configuration
agent_dict = {"main_agent": main_agent_config}
# Define the actions

action_dict = {
    "humanoid_joint_action": HumanoidJointActionConfig()
}
env = init_rearrange_env(agent_dict, action_dict)
_ = env.reset()

2024-02-12 00:13:57,855 Initializing dataset RearrangeDataset-v0
2024-02-12 00:13:57,856 Rearrange task assets are not downloaded locally, downloading and extracting now...
2024-02-12 00:13:57,856 Downloaded and extracted the data.
2024-02-12 00:13:57,858 initializing sim RearrangeSim-v0
[00:14:05:679456]:[Warning]:[Metadata] AttributesManagerBase.h(457)::buildAttrSrcPathsFromJSONAndLoad : <PBR Rendering> : No Glob path result found for `data/fpss/pbr` so unable to load templates from that path.
[00:14:07:147344]:[Warning]:[Metadata] SceneDatasetAttributes.cpp(107)::addNewSceneInstanceToDataset : Dataset : 'hssd-hab-articulated-uncluttered' : Lighting Layout Attributes 'data/fpss/scenes-articulated-uncluttered/102817140.scene_instance.json' specified in Scene Attributes but does not exist in dataset, so creating default.
[00:14:07:520839]:[Warning]:[Sim] Simulator.cpp(594)::instanceStageForSceneAttributes : The active scene does not contain semantic annotations : activeSemanticSceneID_

Renderer: Quadro GV100/PCIe/SSE2 by NVIDIA Corporation
OpenGL version: 4.6.0 NVIDIA 535.54.03
Using optional features:
    GL_ARB_vertex_array_object
    GL_ARB_separate_shader_objects
    GL_ARB_robustness
    GL_ARB_texture_storage
    GL_ARB_texture_view
    GL_ARB_framebuffer_no_attachments
    GL_ARB_invalidate_subdata
    GL_ARB_texture_storage_multisample
    GL_ARB_multi_bind
    GL_ARB_direct_state_access
    GL_ARB_get_texture_sub_image
    GL_ARB_texture_filter_anisotropic
    GL_KHR_debug
    GL_KHR_parallel_shader_compile
    GL_NV_depth_buffer_float
Using driver workarounds:
    no-forward-compatible-core-context
    nv-egl-incorrect-gl11-function-pointers
    no-layout-qualifiers-on-old-glsl
    nv-zero-context-profile-mask
    nv-implementation-color-read-format-dsa-broken
    nv-cubemap-inconsistent-compressed-image-size
    nv-cubemap-broken-full-compressed-image-query
    nv-compressed-block-size-in-bits


In [5]:
def to_pointcloud(obs, cam_matrix):
    # Go from depth and rgb and camera params to pointcloud
    # plt.figure()
    # plt.imshow(obs["head_rgb"])
    cam_matrix[1, :] *= -1
    cam_matrix[2, :] *= -1
    fov = float(env.sim.agents[0]._sensors["head_rgb"].hfov) * np.pi / 180
    metric_depth = obs["head_depth"]
    height, width = metric_depth.shape
    
    fs = width / (2* np.tan(fov / 2.))
    pointcloud = np.ones((height, width, 4))
    depth = metric_depth / fs
    pointcloud[:,:,0] = (np.arange(width)[None, ...] - width/2) * depth
    pointcloud[:,:,1] = (np.arange(height)[..., None] - height/2) * depth
    pointcloud[:,:,2] = -metric_depth

    # pointcloud = pointcloud[..., [2,1,0, 3]]
    # pointcloud = np.concatenate([pointcloud[..., [2]], pointcloud[..., [1]], pointcloud[..., [0]], pointcloud[..., [3]]], 2)
    pco = pointcloud.copy()
    
    pointcloud = (cam_matrix @ pointcloud.reshape(-1, 4).transpose()).transpose()
    pointcloud = pointcloud.reshape(-1, 4)

    colors = obs["head_rgb"][:,:,:3].reshape(-1, 3)/255.
    # .reshape(height, width, 3)
    return pointcloud, colors, pco

def to_pointclouds(observations, camera_params):
    pcs, cols = [], []
    for i in range(len(observations)):
        pc, col, pco = to_pointcloud(observations[i], camera_params[i])
        pcs.append(pc)
        cols.append(col)
    return np.concatenate(pcs, 0), np.concatenate(cols, 0), pco


env.reset()

# Set the initial agent position and rotation
env.sim.agents_mgr[0].articulated_agent.base_rot = -np.pi / 2
env.sim.agents_mgr[0].articulated_agent.base_pos = mn.Vector3(-3,0,2)

# Save all the cameras
cameras_all = []
observations_all = []
current_angle = np.pi

# Initial camera transformation
init_transform = mn.Matrix4(env.sim.agents[0]._sensors["head_rgb"].node.transformation)
for _ in range(10):
    env.sim.agents_mgr[0].articulated_agent.base_rot = current_angle
    env.sim.agents[0]._sensors["head_rgb"]
    current_angle += np.pi/5

    env.sim.step({})
    observations = env.sim.get_sensor_observations()
    observations_all.append(observations)
    
    agent_node = env.sim._default_agent.scene_node
    

    cam_trans = np.array(env.sim.agents[0]._sensors["head_rgb"].render_camera.camera_matrix.inverted())

    cameras_all.append(cam_trans)

pc, colors, pco = to_pointclouds(observations_all, cameras_all)
pc = pc[:,:3]

import open3d as o3d
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(pc)
pcd.colors = o3d.utility.Vector3dVector(colors)
o3d.io.write_point_cloud("./data.ply", pcd)
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import proj3d

# fig = plt.figure(figsize=(8, 8))
# ax = fig.add_subplot(111, projection='3d')
# print(pc.shape)
# stride = 4
# ax.scatter(pc[::stride, 0], pc[::stride, 2], pc[::stride, 1], c=colors[::stride])
# plt.show()